In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
# Recommendation to leave the logging config like this, otherwise you'll be flooded with unnecessary info
import logging
logging.basicConfig(level=logging.WARNING, format='%(levelname)s:%(filename)s:%(message)s')  

dotLog = logging.getLogger(name="src.ui.dotter")
dotLog.setLevel(logging.DEBUG)
entityLog = logging.getLogger(name="src.model.entity")
# entityLog.setLevel(logging.DEBUG)
ttLog = logging.getLogger(name="src.simulator.transitiontime")
# ttLog.setLevel(logging.DEBUG)
z3calcLog = logging.getLogger(name="src.simulator.z3calculator")
# z3calcLog.setLevel(logging.DEBUG)
to_z3Log = logging.getLogger(name="src.simulator.to_z3")
# to_z3Log.setLevel(logging.DEBUG)
ctccLog = logging.getLogger(name="src.simulator.conditiontimedchangecalculator")
# ctccLog.setLevel(logging.DEBUG)
z3ctccLog = logging.getLogger(name="src.simulator.z3conditionchangecalculator")
# z3ctccLog.setLevel(logging.DEBUG)
simLog = logging.getLogger(name="src.simulator.simulator")
simLog.setLevel(logging.DEBUG)

# import all model concepts
from src.model import *

# import the simulator
from src.simulator.simulator import Simulator
from src.simulator.z3simulator import Z3Simulator


# import a library that can visualise our systems
from src.ui import dotter

In [ ]:
m = Resource("m", REAL)
mps = Resource("m/s", REAL)
restitution = Resource("factor", REAL)

import matplotlib.pyplot as plt

def plot_traces(sim):
    store = sim.traces.datastore

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    # adjust the subplot height & width according to kwargs
    fig.set_figwidth(15)
    fig.set_figheight(7)

    for port_entity, time_value_map in store.items():
        if isinstance(port_entity, Port) and port_entity._name in ["vy", "y", "x", "vx"]:
            ax.plot(time_value_map.keys(), time_value_map.values(), label=port_entity._name)
    ax.axhline(y=0, color='k',linewidth=.5)
    ax.axvline(x=0, color='k',linewidth=.5)

    plt.legend(loc='best')
    pass


In [ ]:
class Mass(Entity):
    
    e = Local(restitution, 0.9)  # restitution factor
    
    x = Local(m, 0)  # the position (init should be an input param)
    y = Local(m, 3)  # the height of the table (should be an input param)
    
    vx = Local(mps, 3)  # this should be an input param
    vy = Local(mps, 0)  # this should be an input param
    
    ax = Local(mps, 0)
    ay = Local(mps, 0)
    
    L = Input(m, 7)  # table length
    H = Input(m, 3)  # table height

    
    on_table = current = State()
    falling = State()  # downward movement
    bouncing = State()  # upward movement
    
    fall_off_table = Transition(source=on_table, target=falling, guard=(lambda self: self.x.value == self.L.value))
    bounce = Transition(source=falling, target=bouncing, guard=(lambda self: abs(self.y.value) <= 0.1 * 10**-5))  # instead of equality or lower-equals we make sure that it is in the region around the 0 point (i.e. the absolute value is smaller than an epsilon).
    fall   = Transition(source=bouncing, target=falling, guard=(lambda self: abs(self.vy.value) <= 0.1 * 10**-5 and abs(self.y.value) >= 0.1 * 10**-5 ))  #  This is to fix the "overshooting" problem of equality but also the unsatisfiability of a standard-inequality (<= 0)
    
    """ X value """
    
    @update(state=[on_table, falling, bouncing], target=x)
    def update_x(self, dt):
        return self.x.value + self.vx.value * dt
    
        
    @update(state=bouncing, target=vx)
    def update_vx_bounce_up(self, dt):
        # on every bounce we reduce the value using the restitution factor
        if abs(self.y.value) <= 0.1 * 10**-5 and self.vy.value < 0:  # this means we're near the ground, and we haven't changed to upwards movement yet. good time to apply restitution factor
            return self.vx.value * self.e.value        
        else:
            return self.vx.value

    """ Y value """
    
    @update(state=[on_table, falling], target=y)
    def update_y(self, dt):
        # speed is still zero, don't change
        if abs(self.vy.value) <= 0.1 * 10**-5:
            # print("UNCHANGED", "dt", dt, "vy", self.vy.value, "ay", self.ay.value)
            return self.y.value
        else:
            # z3 will calculate based on the conception that vy is the value at the end, but we need the one at the start for this!
            agv_v = (self.vy.value + self.vy.value - self.ay.value * dt) / 2.0 # velocity at the end is the velocity after adding 9.81 m/s times the 
            # print("CHANGE", "dt", dt, "vy", self.vy.value, "ay", self.ay.value, "new y: ", self.y.value + agv_v * dt, "avg_v", agv_v)
            return self.y.value + agv_v * dt  # s = (v0+v_end)/2*t ; s is the traversed distance  (v_end+v0)/2*t 
    
    @update(state=[bouncing,falling], target=vy)
    def update_vy_falling(self, dt):
        return self.vy.value + self.ay.value * dt

    @update(state=bouncing, target=vy)
    def update_vy_bouncing(self, dt):
        if abs(self.y.value) <= 0.1 * 10**-5 and self.vy.value < 0:  # this means we're near the ground, and we haven't changed to upwards movement yet. good time to apply restitution factor
            return self.vy.value * self.e.value  * -1 # use restitution and inverse force
        else:
            return self.vy.value + self.ay.value * dt
        
    @update(state=[falling, bouncing], target=ay)
    def update_ay_falling(self, dt):
        return -9.81
    
mass = Mass()
sim = Simulator(mass, plotter=dotter)
sim.stabilise()
sim.advance(7)
plot_traces(sim)

In [ ]:
mass = Mass()
sim = Simulator(mass, plotter=dotter)
sim.stabilise()
sim.advance(7)
#[sim.advance(.1) for _ in range(70)]

In [ ]:
# mass = Mass()
# sim = Simulator(mass, plotter=dotter)
# sim.stabilise()
# sim.advance(2)
# [sim.advance(.1) for _ in range(10)]
# sim.plot()

In [ ]:
# sim.advance(0.1)
# sim.plot()
# # sim.advance(0.1)
# sim.plot()
# sim.advance(0.1)
# sim.plot()
# sim.advance(0.1)
# sim.plot()

In [ ]:
# sim.traces.plot(*get_ports(sim.entity), height=3, width=15)